In [1]:
import trainer
import json
import tensorflow as tf
import numpy as np
import re

In [2]:
with open('class2idx.json', 'r') as inp:
    class2idx = json.load(inp)
with open('word2idx.json', 'r') as inp:
    word2idx = json.load(inp)

In [3]:
train_data = np.load('train.npy')
valid_data = np.load('valid.npy')
test_data = np.load('test.npy')

In [4]:
VERSION = 5
params = dict(
    model_configs = {
   "rnn_layers":[
          {
             "units": 1200,
             "input_size":400,
             "drop_i": 0.01,
             "drop_w": 0.1,
             "drop_o": 0.01
          },
          {
             "units": 1200,
             "input_size": 1200,
             "drop_w": 0.1,
             "drop_o": 0.01
          },
          {
             "units": 400,
             "input_size": 1200,
             "drop_o": 0.2,
             "drop_w": 0.1
          }
       ],
       "vocab_size": len(word2idx),
       "drop_e": 0.0
    },
    optimizer = tf.train.AdamOptimizer,
    learning_rate = 1e-3,
    decay_freq = 100000,
    decay_rate = 0.9,
    wdecay = 1.2e-6,
    alpha = 0.0,
    beta = 0.0,
    clip_norm = 1.0,
    bptt = 200,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)


# In[12]:


my_trainer = trainer.Trainer(**params)

my_trainer.logger.info('Trainer params {}'.format(params))
# In[13]:


tf.reset_default_graph()
my_trainer.build()

Trainer params {'model_configs': {'rnn_layers': [{'units': 1200, 'input_size': 400, 'drop_i': 0.01, 'drop_w': 0.1, 'drop_o': 0.01}, {'units': 1200, 'input_size': 1200, 'drop_w': 0.1, 'drop_o': 0.01}, {'units': 400, 'input_size': 1200, 'drop_o': 0.2, 'drop_w': 0.1}], 'vocab_size': 380, 'drop_e': 0.0}, 'optimizer': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.001, 'decay_freq': 100000, 'decay_rate': 0.9, 'wdecay': 1.2e-06, 'alpha': 0.0, 'beta': 0.0, 'clip_norm': 1.0, 'bptt': 200, 'use_ema': True, 'save_freq': 1000, 'log_path': '5/logs', 'train_summary_dir': '5/train_summary/', 'test_summary_dir': '5/test_summary/', 'checkpoint_dir': '5/checkpoints/'}
[<tf.Variable 'LanguageModel/embedding_weight:0' shape=(380, 400) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/W:0' shape=(400, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/U:0' shape=(1200, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/bias:0'

INFO:tensorflow:Restoring parameters from 5/checkpoints/train/model.cpkt-111224


Restoring parameters from 5/checkpoints/train/model.cpkt-111224
Restore variables: [<tf.Variable 'LM_Trainer/global_step:0' shape=() dtype=int32_ref>, <tf.Variable 'LanguageModel/decoder_b/ExponentialMovingAverage:0' shape=(380,) dtype=float32_ref>, <tf.Variable 'LanguageModel/decoder_b:0' shape=(380,) dtype=float32_ref>, <tf.Variable 'LanguageModel/embedding_weight/ExponentialMovingAverage:0' shape=(380, 400) dtype=float32_ref>, <tf.Variable 'LanguageModel/embedding_weight:0' shape=(380, 400) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/U/ExponentialMovingAverage:0' shape=(1200, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/U:0' shape=(1200, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/W/ExponentialMovingAverage:0' shape=(400, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/W:0' shape=(400, 4800) dtype=float32_ref>, <tf.Variable 'LanguageModel/lstm_fused_cell/bias/ExponentialMovingAverage:0' shap

In [5]:
classifier_configs = dict(
    hiddens = [
        dict(
            filters=100,
            kernel_size=5,
            drop_i=0.5,
            drop_o=0.5,
            l2_reg=2e-3
        ),
        dict(
            filters=100,
            kernel_size=5,
            dilation_rate=2,
            drop_o=0.5,
            l2_reg=2e-3
        ),
        dict(
            filters=100,
            kernel_size=5,
            dilation_rate=2,
            drop_o=0.5,
            l2_reg=2e-3
        )
    ], 
    k = 3
)
my_trainer.add_classifier(num_classes=len(class2idx), classifier_configs=classifier_configs)

Classifier configs: {'hiddens': [{'filters': 100, 'kernel_size': 5, 'drop_i': 0.5, 'drop_o': 0.5, 'l2_reg': 0.002}, {'filters': 100, 'kernel_size': 5, 'dilation_rate': 2, 'drop_o': 0.5, 'l2_reg': 0.002}, {'filters': 100, 'kernel_size': 5, 'dilation_rate': 2, 'drop_o': 0.5, 'l2_reg': 0.002}], 'k': 3}
['Classifier_0/conv1d_0/kernel:0', 'Classifier_0/conv1d_0/bias:0', 'Classifier_0/batch_normalization/gamma:0', 'Classifier_0/batch_normalization/beta:0', 'Classifier_0/batch_normalization/moving_mean:0', 'Classifier_0/batch_normalization/moving_variance:0', 'Classifier_0/conv1d_1/kernel:0', 'Classifier_0/conv1d_1/bias:0', 'Classifier_0/batch_normalization_1/gamma:0', 'Classifier_0/batch_normalization_1/beta:0', 'Classifier_0/batch_normalization_1/moving_mean:0', 'Classifier_0/batch_normalization_1/moving_variance:0', 'Classifier_0/conv1d_2/kernel:0', 'Classifier_0/conv1d_2/bias:0', 'Classifier_0/batch_normalization_2/gamma:0', 'Classifier_0/batch_normalization_2/beta:0', 'Classifier_0/batch

In [6]:
def generator(data, bsz, num_class, noise=0.0, shuffle=True, max_length = 1000000000000):
    if shuffle:
        data = np.random.permutation(data)
    i = 0
    while i < len(data):
        next_bsz = min(bsz, len(data) - i)
        next_batch = data[i:i+next_bsz]
        seq_lens = np.array([min(max_length, len(x)) for x, _ in next_batch], dtype=np.int32)
        next_x = np.zeros(shape=(next_bsz, max(seq_lens)), dtype=np.int32)
        for j, ((x, _), xl) in enumerate(zip(next_batch, seq_lens)):
            next_x[j, :xl] = x[:xl]
        next_x = np.transpose(next_x, axes=[1, 0])
        next_y = [
            np.zeros(shape=(next_bsz, 2), dtype=np.float32) for _ in range(num_class)
        ]
        for j, (_, c) in enumerate(next_batch):
            for ci in range(num_class):
                if ci == c:
                    next_y[ci][j][1] = 1 - noise
                    next_y[ci][j][0] = noise
                else:
                    next_y[ci][j][0] = 1 - noise
                    next_y[ci][j][1] = noise
        yield next_x, next_y, seq_lens
        i += next_bsz

In [ ]:
for _ in range(5):
    train_gen = generator(train_data, 100, len(class2idx), 0.0, True, 400)
    val_gen = generator(valid_data, 100, len(class2idx), 0.0, False, 400)
    my_trainer.classifier_train_dev_loop(train_gen, val_gen)

Step    1: loss 12.81320, time 03.28
Step    2: loss 9.03220, time 00.77
Step    3: loss 9.57012, time 00.77
Step    4: loss 8.38230, time 00.77
Step    5: loss 7.64284, time 00.77
Step    6: loss 8.08059, time 00.77
Step    7: loss 6.98950, time 00.77
Step    8: loss 8.06201, time 00.77
Step    9: loss 7.34099, time 00.77
Step   10: loss 7.73129, time 00.77
Step   11: loss 7.59348, time 00.78
Step   12: loss 7.73335, time 00.79
Step   13: loss 6.44204, time 00.79
Step   14: loss 7.96272, time 00.79
Step   15: loss 7.68232, time 00.78
Step   16: loss 6.52353, time 00.78
Step   17: loss 6.92045, time 00.77
Step   18: loss 6.08749, time 00.79
Step   19: loss 7.18192, time 00.79
Step   20: loss 6.27559, time 00.79
Step   21: loss 7.19957, time 00.79
Step   22: loss 6.29389, time 00.78
Step   23: loss 6.10088, time 00.80
Step   24: loss 5.71332, time 00.79
Step   25: loss 5.91462, time 00.79
Step   26: loss 6.54303, time 00.82
Step   27: loss 5.98207, time 00.79
Step   28: loss 5.40517, ti

Step  229: loss 2.32105, time 00.78
Step  230: loss 2.34105, time 00.78
Step  231: loss 2.34823, time 00.78
Step  232: loss 2.35511, time 00.78
Step  233: loss 2.29582, time 00.78
Step  234: loss 2.41042, time 00.78
Step  235: loss 2.23152, time 00.78
Step  236: loss 2.32175, time 00.78
Step  237: loss 2.43583, time 00.78
Step  238: loss 2.33133, time 00.78
Step  239: loss 2.26305, time 00.78
Step  240: loss 2.32147, time 00.78
Step  241: loss 2.45133, time 00.78
Step  242: loss 2.39264, time 00.78
Step  243: loss 2.43063, time 00.78
Step  244: loss 2.32183, time 00.78
Step  245: loss 2.28183, time 00.78
Step  246: loss 2.28861, time 00.78
Step  247: loss 2.45089, time 00.78
Step  248: loss 2.35794, time 00.78
Step  249: loss 2.29117, time 00.78
Step  250: loss 2.16284, time 00.78
Step  251: loss 2.34560, time 00.78
Step  252: loss 2.20883, time 00.78
Step  253: loss 2.31659, time 00.78
Step  254: loss 2.48436, time 00.78
Step  255: loss 2.40806, time 00.78
Step  256: loss 2.14958, tim

Step  410: loss 2.24323, time 00.78
Step  411: loss 2.03518, time 00.78
Step  412: loss 2.24604, time 00.78
Step  413: loss 2.12363, time 00.78
Step  414: loss 2.29219, time 00.78
Step  415: loss 2.15361, time 00.78
Step  416: loss 2.39748, time 00.78
Step  417: loss 2.12682, time 00.78
Step  418: loss 2.17833, time 00.78
Step  419: loss 2.41134, time 00.78
Step  420: loss 2.17703, time 00.78
Step  421: loss 2.36053, time 00.78
Step  422: loss 2.10723, time 00.78
Step  423: loss 2.18072, time 00.79
Step  424: loss 2.27325, time 00.78
Step  425: loss 2.31062, time 00.78
Step  426: loss 2.22602, time 00.78
Step  427: loss 2.15929, time 00.79
Step  428: loss 2.16113, time 00.78
Step  429: loss 2.19535, time 00.78
Step  430: loss 2.16268, time 00.78
Step  431: loss 2.16835, time 00.79
Step  432: loss 2.12362, time 00.80
Step  433: loss 2.10344, time 00.80
Step  434: loss 2.09710, time 00.81
Step  435: loss 2.18434, time 00.79
Step  436: loss 2.15428, time 00.78
Step  437: loss 2.35985, tim

Step  591: loss 2.15176, time 00.81
Step  592: loss 2.06471, time 00.83
Step  593: loss 2.08108, time 00.81
Step  594: loss 2.12249, time 00.80
Step  595: loss 2.03235, time 00.81
Step  596: loss 1.85455, time 00.81
Step  597: loss 1.90647, time 00.82
Step  598: loss 1.82505, time 00.80
Step  599: loss 1.89458, time 00.81
Step  600: loss 2.06514, time 00.80
Step  601: loss 2.01493, time 00.78
Step  602: loss 1.67609, time 00.78
Step  603: loss 2.06158, time 00.78
Step  604: loss 1.99244, time 00.78
Step  605: loss 1.92865, time 00.78
Step  606: loss 1.89609, time 00.78
Step  607: loss 1.98338, time 00.78
Step  608: loss 1.71981, time 00.78
Step  609: loss 1.93727, time 00.78
Step  610: loss 1.96130, time 00.78
Step  611: loss 1.99006, time 00.79
Step  612: loss 1.95134, time 00.78
Step  613: loss 2.07106, time 00.79
Step  614: loss 2.02046, time 00.78
Step  615: loss 2.02679, time 00.79
Step  616: loss 1.93850, time 00.78
Step  617: loss 1.98558, time 00.78
Step  618: loss 2.05011, tim

Step  819: loss 1.52862, time 00.80
Step  820: loss 1.63110, time 00.80
Step  821: loss 1.62371, time 00.82
Step  822: loss 1.57116, time 00.78
Step  823: loss 1.74438, time 00.79
Step  824: loss 1.60686, time 00.78
Step  825: loss 1.67128, time 00.78
Step  826: loss 1.51286, time 00.78
Step  827: loss 1.61175, time 00.78
Step  828: loss 1.86049, time 00.79
Step  829: loss 1.56298, time 00.79
Step  830: loss 1.66946, time 00.78
Step  831: loss 1.74453, time 00.79
Step  832: loss 1.76421, time 00.78
Step  833: loss 1.79965, time 00.79
Step  834: loss 1.58927, time 00.78
Step  835: loss 1.59168, time 00.78
Step  836: loss 1.85803, time 00.78
Step  837: loss 1.65099, time 00.78
Step  838: loss 1.67474, time 00.78
Step  839: loss 1.49890, time 00.78
Step  840: loss 1.66144, time 00.78
Step  841: loss 1.67451, time 00.79
Step  842: loss 1.71483, time 00.79
Step  843: loss 1.52363, time 00.78
Step  844: loss 1.86510, time 00.78
Step  845: loss 1.59668, time 00.78
Step  846: loss 1.67975, tim

In [ ]:
test_gen = generator(test_data, 30, len(class2idx), 0.0, False, 400)
val_gen = generator(valid_data, 30, len(class2idx), 0.0, False, 400)

In [ ]:
my_trainer.classifier_evaluate_step(test_gen)

In [ ]:
my_trainer.classifier_evaluate_step(val_gen)

In [ ]:
def predict(text, max_len=400):
    text = text[:min(max_len, len(text))]
    text = re.sub('\d+','N',re.sub('\W+','',re.sub('[ ]+','_',re.sub('[\n\r]+','_nl_', text.lower()))))
    print(text)
    ids = np.array([word2idx[c] for c in text.replace('_nl_', '\n') if c in word2idx], dtype=np.int32)
    if len(ids) < 20:
        t = np.zeros(shape=(20), dtype=np.int32)
        t[:len(ids)] = ids
        ids = t
    seq_lens = len(ids)
    classifiers = my_trainer.test_classifiers
    inputs = np.transpose(np.expand_dims(ids, axis=0), [1, 0])
    fd = {
        my_trainer.model_test.inputs: inputs,
        my_trainer.model_test.seq_lens: np.array([seq_lens], dtype=np.int32),
        my_trainer.model_test.reset_state: True
    }
    pred_ys = my_trainer.session.run([c.out_prob for c in classifiers], feed_dict=fd)
    return {
        k: v[0][1] for k, v in zip(class2idx, pred_ys)
    }

In [ ]:
idx2word = {k: v for v, k in word2idx.items()}

In [ ]:
a = [''.join(idx2word[x] for x in y) for y, _ in test_data]

In [ ]:
for i, d in enumerate(a):
    print(d)
    print(predict(d))
    print(test_data[i][1])
    print('-' * 90)

In [ ]:
predict('cách_giảm_cân_khoa_học_nhất_để_giảm_cân_là_tập_thể_dục_và_ăn_kiêng_hợp_lý_tuy_nhiên_nếu_không_có_chế_độ_ăn_uống_hợp_lý_sẽ_khiến_mọi_công_sức_tập_luyện_trở_nên_vô_ích_hoặc_chúng_ta_sẽ_không_đủ_năng_lượng_để_cơ_thể_phục_hồi_sau_tập_luyện_vây_chúng_ta_ăn_gì_sau_khi_tập_thể_dục_để_giảm_cân_')

In [ ]:
predict('tông_màu_pastel_nhẹ_nhàng_không_chỉ_tạo_nên_không_khí_vui_tươi_của_mùa_xuân_mà_còn_là_niềm_cảm_hứng_bất_tận_cho_những_tấm_hình_sống_ảo_của_cộng_đồng_mạng_cùng_điểm_qua_những_địa_điểm_thú_vị_cho_các_fan_của_pastel_nào')